# 15. TF2 API 개요
----------------------------------------
### 목차

* 2-5. TensorFlow API로 모델 구성하기
  + Sequential API
  + Functional API
  + Subclassing API
* 6-8. TensorFlow API 모델 작성 및 학습하기
  + Sequential API
  + Function API
  + Subclassing API
* 9. GradientTape의 활용
  + `model.fit()`에서 벗어나서 커스텀 학습을 진행시킬 수 있습니다.

### 학습목표
* Tensorflow V2의 개요와 특징을 파악한다.
* Tensorflow V2의 3가지 주요 API 구성 방식을 이해하고 활용할 수 있다.
* GradientTape를 활용해 보고 좀 더 로우 레벨의 딥러닝 구현 방식을 이해한다.

------------------------------
## 15-2. TensorFlow2 API로 모델 구성하기


* Sequential Model은 하나의 입력(input)과 출력(output)으로 모델을 구성한다.
* Functional API를 통해 다중 입력/출력을 가지는 모델을 구성할 수 있다.

## 15-3. Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용



In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = tf.expand_dims(x_train, axis=3)
x_test = tf.expand_dims(x_test, axis=3)

print(x_train.shape, x_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [2]:
# Sequential Model을 구성해주세요.
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(64, 3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4,738,826
Trainable params: 4,738,826
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)

model.evaluate(x_test,  y_test, verbose=2)

 101/1875 [>.............................] - ETA: 4:18 - loss: 0.4917 - accuracy: 0.8518

KeyboardInterrupt: ignored

## 15-4. Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.expand_dims(x_train, axis = 3)
x_test = tf.expand_dims(x_test, axis = 3)

print(len(x_train), len(x_test))

In [ ]:
inputs = keras.layers.Input(shape=x_train.shape[1:])
x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10)(x)

model = keras.Model(inputs=inputs, outputs=predictions)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test,  y_test, verbose=2)

## 15-5. Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.expand_dims(x_train, axis = 3)
x_test = tf.expand_dims(x_test, axis = 3)

print(len(x_train), len(x_test))

60000 10000


In [ ]:
# Subclassing을 활용한 Model을 구성해주세요.

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

    def summary(self):
        x = keras.Input(shape = (28,28,1))
        model = keras.Model(inputs=[x], outputs = self.call(x))
        return model.summary()


model = CustomModel()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4,738,826
Trainable params: 4,738,826
Non-train

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/2
1875/1875 [==============================] - 281s 149ms/step - loss: 0.1112 - accuracy: 0.9660
Epoch 2/2
1875/1875 [==============================] - 266s 142ms/step - loss: 0.0364 - accuracy: 0.9889
313/313 - 9s - loss: 0.0378 - accuracy: 0.9880 - 9s/epoch - 30ms/step


[0.037776168435811996, 0.9879999756813049]

## 15-8. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용


In [ ]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

169001437/169001437 [==============================] - 7s 0us/step
50000 10000


In [ ]:
# Subclassing을 활용한 Model을 구성해주세요.

# 여기에 모델을 구성해주세요
class CustomModel_cifar100(keras.Model):
    def __init__(self):
        super().__init__()
        self.Conv1 = tf.keras.layers.Conv2D(16,3,activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D(2)
        self.Conv2 = tf.keras.layers.Conv2D(32,3,activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D(2)
        self.Flat = tf.keras.layers.Flatten()
        self.FC1 = tf.keras.layers.Dense(256,activation='relu')
        self.FC2 = tf.keras.layers.Dense(100,activation='softmax')
    
    def call(self,x):
        x = self.Conv1(x)
        x = self.pool1(x)
        x = self.Conv2(x)
        x = self.pool2(x)
        x = self.Flat(x)
        x = self.FC1(x)
        return self.FC2(x) 

    def summary(self):
        x = keras.Input(shape = (32,32,3))
        model = keras.Model(inputs=[x], outputs = self.call(x))
        return model.summary()


model = CustomModel_cifar100()

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1152)              0   

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 50s 32ms/step - loss: 2.6909 - accuracy: 0.3269
Epoch 2/5
1563/1563 [==============================] - 49s 31ms/step - loss: 2.4956 - accuracy: 0.3661
Epoch 3/5
1563/1563 [==============================] - 49s 31ms/step - loss: 2.3321 - accuracy: 0.3993
Epoch 4/5
1563/1563 [==============================] - 50s 32ms/step - loss: 2.1924 - accuracy: 0.4303
Epoch 5/5
1563/1563 [==============================] - 49s 32ms/step - loss: 2.0724 - accuracy: 0.4548
313/313 - 3s - loss: 2.6542 - accuracy: 0.3536 - 3s/epoch - 9ms/step


[2.6542258262634277, 0.35359999537467957]